In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import logging
from datetime import datetime

# Configuración de los logs
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
logger = logging.getLogger(__name__)

class BaseStrategy:
    def __init__(self):
        pass

    def execute_trade(self, signal):
        raise NotImplementedError("Este método debe ser implementado por estrategias específicas.")

    def get_signal(self, market_data):
        raise NotImplementedError("Este método debe ser implementado por estrategias específicas.")

class ThresholdStrategy(BaseStrategy):
    def __init__(self, buy_threshold=1.06451, sell_threshold=1.074):
        self.buy_threshold = buy_threshold
        self.sell_threshold = sell_threshold

    def get_signal(self, market_data):
        market_data['signal'] = 0
        tolerance = 0.0005 # Ajusta la tolerancia según lo necesario
        market_data['buy_signal'] = ((market_data['close'] >= self.buy_threshold - tolerance) & 
                                     (market_data['close'] <= self.buy_threshold + tolerance)).astype(int)
        market_data['sell_signal'] = ((market_data['close'] >= self.sell_threshold - tolerance) & 
                                      (market_data['close'] <= self.sell_threshold + tolerance)).astype(int)
        return market_data

    def execute_trade(self, signal):
        if signal == 1:
            print("Ejecutar compra")
        elif signal == -1:
            print("Ejecutar venta")
        else:
            print("Mantener posición")

def load_and_process_data(file_path, chunksize=10000):
    try:
        logger.info(f"Cargando y procesando datos desde {file_path} en bloques de {chunksize} filas...")

        # DataFrame final para almacenar los resultados procesados
        all_data = pd.DataFrame()
        error_count = 0  # Contador de filas con errores

        # Leer el archivo CSV en bloques
        for chunk in pd.read_csv(
            file_path,
            header=0,  # Asumimos que el archivo CSV tiene encabezados
            names=["DateTime", "Bid", "Ask", "Volume"],
            dtype={"DateTime": str, "Bid": float, "Ask": float, "Volume": int},  # Especificar tipos de datos
            chunksize=chunksize,
            low_memory=False
        ):
            logger.info("Procesando un nuevo bloque...")

            # Convertir la columna 'DateTime' a tipo datetime
            chunk['DateTime'] = pd.to_datetime(chunk['DateTime'], format='%Y%m%d %H:%M:%S.%f', errors='coerce')

            # Filtrar filas con valores inválidos o NaN
            chunk = chunk.dropna()

            # Concatenar el bloque procesado al DataFrame final
            all_data = pd.concat([all_data, chunk], ignore_index=True)

        # Configurar la columna 'DateTime' como índice
        all_data.set_index('DateTime', inplace=True)

        # Agregar la columna 'close' si no existe
        if 'close' not in all_data.columns:
            logger.info("Agregando columna 'close' como el promedio de 'Bid' y 'Ask'")
            all_data['close'] = (all_data['Bid'] + all_data['Ask']) / 2

        # Guardar el DataFrame procesado en un archivo CSV
        processed_file_path = file_path.replace('.csv', '_processed.csv')
        all_data.to_csv(processed_file_path)
        logger.info(f"Datos procesados guardados en {processed_file_path}")

        logger.info("Datos cargados y procesados correctamente.")
        logger.info(f"Total de filas con errores: {error_count}")
        return all_data

    except Exception as e:
        logger.error(f"Error al procesar los datos: {e}")
        return None

def test_strategy(strategy, market_data):
    signals = strategy.get_signal(market_data)

    # Agrupar los datos por horas
    hourly_data = market_data.resample('h').agg({
        'Bid': 'mean',
        'Ask': 'mean',
        'close': 'mean',
        'Volume': 'sum',
        'buy_signal': 'sum',
        'sell_signal': 'sum'
    })

    # Mostrar todas las veces que se hubiese activado la compra y venta 
    sell_signals = hourly_data[hourly_data['sell_signal'] > 0]
    print("Señales de venta activadas:")
    print(sell_signals[['close', 'sell_signal']])
    buy_signals = hourly_data[hourly_data['buy_signal'] > 0]
    print("Señales de compra activadas:")
    print(buy_signals[['close', 'buy_signal']])

    # Visualizar los datos y las señales
    fig = go.Figure()

    fig.add_trace(go.Scatter(x=hourly_data.index, y=hourly_data['close'], mode='lines+markers', name='Close Price'))
    fig.add_trace(go.Scatter(x=buy_signals.index, y=buy_signals['close'], mode='markers', marker=dict(color='green', size=10), name='Buy Signal'))
    fig.add_trace(go.Scatter(x=sell_signals.index, y=sell_signals['close'], mode='markers', marker=dict(color='red', size=10), name='Sell Signal'))

    fig.update_layout(title='Señales de Compra y Venta por Horas', xaxis_title='Fecha', yaxis_title='Precio de Cierre')
    fig.show()

# Ruta al archivo CSV (actualiza esta ruta según tu sistema)
file_path = '/Users/daniel/Desktop/bot/trading-bot/src/data/data.csv'

# Procesar el archivo
data = load_and_process_data(file_path, chunksize=100000)

if data is not None:
    logger.info(data.info())
    print(data.head())

    # Crear una estrategia y probarla
    strategy = ThresholdStrategy()

    # Crear datos simulados para pruebas si no hay columna 'close'
    if 'close' not in data.columns:
        data['close'] = (data['Bid'] + data['Ask']) / 2

    test_strategy(strategy, data)
else:
    logger.error("No se generó ningún DataFrame procesado.")

2025-01-30 09:39:12,406 - INFO - Cargando y procesando datos desde /Users/daniel/Desktop/bot/trading-bot/src/data/data.csv en bloques de 100000 filas...
2025-01-30 09:39:12,531 - INFO - Procesando un nuevo bloque...
2025-01-30 09:39:12,667 - INFO - Procesando un nuevo bloque...
2025-01-30 09:39:12,796 - INFO - Procesando un nuevo bloque...
2025-01-30 09:39:12,954 - INFO - Procesando un nuevo bloque...
2025-01-30 09:39:13,085 - INFO - Procesando un nuevo bloque...
2025-01-30 09:39:13,222 - INFO - Procesando un nuevo bloque...
2025-01-30 09:39:13,369 - INFO - Procesando un nuevo bloque...
2025-01-30 09:39:13,520 - INFO - Procesando un nuevo bloque...
2025-01-30 09:39:13,763 - INFO - Procesando un nuevo bloque...
2025-01-30 09:39:13,912 - INFO - Procesando un nuevo bloque...
2025-01-30 09:39:14,066 - INFO - Procesando un nuevo bloque...
2025-01-30 09:39:14,216 - INFO - Procesando un nuevo bloque...
2025-01-30 09:39:14,359 - INFO - Procesando un nuevo bloque...
2025-01-30 09:39:14,510 - IN

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 20657045 entries, 2024-01-15 02:00:00.287000 to 2025-01-15 01:59:57.014000
Data columns (total 4 columns):
 #   Column  Dtype  
---  ------  -----  
 0   Bid     float64
 1   Ask     float64
 2   Volume  int64  
 3   close   float64
dtypes: float64(3), int64(1)
memory usage: 788.0 MB
                             Bid      Ask   Volume     close
DateTime                                                    
2024-01-15 02:00:00.287  1.09451  1.09453  4500000  1.094520
2024-01-15 02:00:00.390  1.09450  1.09453  5400000  1.094515
2024-01-15 02:00:00.645  1.09451  1.09453   900000  1.094520
2024-01-15 02:00:01.050  1.09450  1.09453  4500000  1.094515
2024-01-15 02:00:02.059  1.09451  1.09453  3600000  1.094520


/var/folders/9d/96cnlc1x6zxcqc7gf9dr489c0000gn/T/ipykernel_5327/2141992094.py:96: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  hourly_data = market_data.resample('H').agg({


Señales de venta activadas:
                        close  sell_signal
DateTime                                  
2024-02-05 15:00:00  1.074490         1955
2024-02-05 16:00:00  1.074083         6593
2024-02-05 17:00:00  1.072991         1473
2024-02-05 18:00:00  1.072912          189
2024-02-05 19:00:00  1.073852         3149
...                       ...          ...
2024-11-07 07:00:00  1.074201         1759
2024-11-07 08:00:00  1.074542         1169
2024-11-08 15:00:00  1.075849           52
2024-11-08 17:00:00  1.074659         2446
2024-11-08 18:00:00  1.072973         1343

[293 rows x 2 columns]
Señales de compra activadas:
                        close  buy_signal
DateTime                                 
2024-04-12 13:00:00  1.065277         895
2024-04-12 14:00:00  1.065366         808
2024-04-12 15:00:00  1.064393        1873
2024-04-12 16:00:00  1.063667        1033
2024-04-12 17:00:00  1.063167         723
...                       ...         ...
2024-11-12 05:00:00  1.0

In [ ]:
import json
import pandas as pd
import sys
import os

# Asegúrate de que el directorio 'src' esté en el mismo nivel que tu cuaderno Jupyter
print(os.getcwd())
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', 'src')))

from strategies.base_strategy import ThresholdStrategy
from data.data_loader import load_and_process_data
from utils.logger import Logger

class TradingBot:
    def __init__(self, config_path):
        self.logger = Logger(__name__)
        self.config = self.load_config(config_path)
        self.strategy = ThresholdStrategy(
            buy_threshold=self.config['strategy']['parameters']['buy_threshold'],
            sell_threshold=self.config['strategy']['parameters']['sell_threshold']
        )
        self.data = None
        self.balance = 1000  # Balance inicial en euros
        self.position = 0  # Posición actual (cantidad de activos)
        self.entry_price = 0  # Precio de entrada de la última operación

    def load_config(self, config_path):
        self.logger.info(f"Cargando configuración desde {config_path}")
        with open(config_path, 'r') as file:
            config = json.load(file)
        return config

    def load_market_data(self):
        processed_file_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'src', 'data', 'data_processed.csv'))
        
        # Verificar si el archivo procesado existe
        if not os.path.exists(processed_file_path):
            self.logger.error(f"El archivo procesado {processed_file_path} no existe.")
            raise ValueError(f"El archivo procesado {processed_file_path} no existe.")
        
        # Cargar el archivo procesado
        self.logger.info(f"Cargando datos procesados desde {processed_file_path}")
        self.data = pd.read_csv(processed_file_path, parse_dates=['DateTime'], index_col='DateTime')
        if self.data is None or self.data.empty:
            self.logger.error("No se pudo cargar los datos del mercado.")
            raise ValueError("No se pudo cargar los datos del mercado.")
        self.logger.info("Datos de mercado cargados correctamente.")

    def run(self):
        if self.data is not None:
            self.logger.info("Ejecutando estrategia de trading")
            signals = self.strategy.get_signal(self.data)
            for index, row in signals.iterrows():
                if row['buy_signal'] == 1 and self.position == 0:
                    self.position = self.balance / row['close']
                    self.entry_price = row['close']
                    self.balance = 0
                    self.logger.info(f"Ejecutar compra a {row['close']}, posición: {self.position}")
                elif row['sell_signal'] == 1 and self.position > 0:
                    self.balance = self.position * row['close']
                    self.position = 0
                    self.logger.info(f"Ejecutar venta a {row['close']}, balance: {self.balance}")

                # Verificar si la pérdida supera los 100 euros
                if self.position > 0 and (self.entry_price - row['close']) * self.position > 100:
                    self.balance = self.position * row['close']
                    self.position = 0
                    self.logger.info(f"Venta forzada a {row['close']} para evitar pérdidas mayores a 100 euros, balance: {self.balance}")
        else:
            self.logger.warning("No hay datos de mercado cargados.")

def test_bot():
    bot = TradingBot(config_path='../config.json')
    bot.load_market_data()
    bot.run()
    print(f"Balance final: {bot.balance}")
    print(f"Posición final: {bot.position}")

# Ejecutar la prueba del bot
test_bot()

/Users/daniel/Desktop/bot/trading-bot/notebooks


2025-01-30 09:44:39,521 - __main__ - INFO - Cargando configuración desde ../config.json
2025-01-30 09:44:39,521 - INFO - Cargando configuración desde ../config.json
2025-01-30 09:44:39,525 - __main__ - INFO - Cargando datos procesados desde /Users/daniel/Desktop/bot/trading-bot/src/data/data_processed.csv
2025-01-30 09:44:39,525 - INFO - Cargando datos procesados desde /Users/daniel/Desktop/bot/trading-bot/src/data/data_processed.csv
2025-01-30 09:45:13,561 - __main__ - INFO - Datos de mercado cargados correctamente.
2025-01-30 09:45:13,561 - INFO - Datos de mercado cargados correctamente.
2025-01-30 09:45:13,569 - __main__ - INFO - Ejecutando estrategia de trading
2025-01-30 09:45:13,569 - INFO - Ejecutando estrategia de trading
